<a href="https://colab.research.google.com/github/ArletteRuiz20/Tool-Enabled-LLM-Agent-Implementation/blob/main/LLM_Agent_Implementation_with_Google_ADK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


- ✅ Install [Agent Development Kit (ADK)](https://google.github.io/adk-docs/)
- ✅ Configure your API key to use the Gemini model
- ✅ Build your first simple agent

### 1.1: Install Google ADK and nest_asyncio

In [ ]:
!pip -q install nest_asyncio
!pip -q install google-adk
!pip -q install -U google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 724.2/724.2 kB 36.9 MB/s eta 0:00:00


### 1.2: Configure your Gemini API Key



In [ ]:
import os

os.environ.pop("GOOGLE_API_KEY", None)

print("cleared.")


cleared.


In [ ]:
from getpass import getpass

def setup_api_key():
    k = os.environ.get("GOOGLE_API_KEY", "")

    if not k or k.startswith("$"):
        k = getpass("Please enter your Google API Key: ").strip()
        os.environ["GOOGLE_API_KEY"] = k

    print("✅ Key captured.")
    print("length =", len(k))
    print("starts_with =", k[:4], "ends_with =", k[-4:])
    print("has_space =", any(c.isspace() for c in k))

setup_api_key()


Please enter your Google API Key: ··········
✅ Key captured.
length = 39
starts_with = AIza ends_with = ni34
has_space = False


In [ ]:
import os
from google import genai

client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])

resp = client.models.generate_content(
    model="gemini-2.5-flash-lite",
    contents="Say Hi welcome:)."
)

print("MODEL SAID:", resp.text)


MODEL SAID: Hi! Welcome! :)


In [ ]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


### 1.4: Configure Retry Options



In [ ]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds) bc we are limited
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

---

## 🤖 Section 2:first AI Agent with ADK



### 2.2 Define agent



In [ ]:
root_agent = Agent(
    name="Arlette's_assistant",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="A simple agent that can answer general questions.",
    instruction="You are a helpful assistant. Use Google Search for current info or if unsure.",
    tools=[google_search],
)

print("✅ Root Agent defined.")

✅ Root Agent defined.


### 2.3 Run agent


In [ ]:
runner = InMemoryRunner(agent=root_agent)

print("✅ Runner created.")

✅ Runner created.


In [ ]:
import nest_asyncio, asyncio
nest_asyncio.apply()

def ask(prompt: str):
    async def _run():
        return await runner.run_debug(prompt)
    return asyncio.get_event_loop().run_until_complete(_run())

In [ ]:
response = ask("What is Agent Development Kit from Google? What languages is the SDK available in?")


 ### Created new session: debug_session_id

User > What is Agent Development Kit from Google? What languages is the SDK available in?
Rachet_assistant > The Google Agent Development Kit (ADK) is a flexible and modular framework designed to simplify the end-to-end development and deployment of AI agents. It provides developers with a set of "Lego blocks" or capabilities that can be assembled to create agents capable of perceiving the world and taking action. The ADK is model-agnostic and deployment-agnostic, and it's built for compatibility with other frameworks, aiming to make agent development more akin to traditional software development. It supports features like multi-agent architectures, model flexibility (including Gemini and other LLMs), and a rich tool ecosystem.

The SDK is available in the following languages:
*   Python
*   Java
*   Go
*   TypeScript


### 🚀 2.5

Agent in action

In [ ]:
response = ask("What's the weather in London?")


 ### Continue session: debug_session_id

User > What's the weather in London?
Rachet_assistant > Currently in London, it is cloudy with a temperature of 8°C (47°F) and 92% humidity. There is a 15% chance of precipitation. The air quality is currently high in pollution and considered unhealthy for sensitive groups.

For the rest of today, expect light rain during the day and cloudy conditions at night, with an 80% chance of rain during the day and a 35% chance of rain at night. The temperature is expected to range between 6°C (43°F) and 9°C (48°F).

Looking ahead, the forecast for the next few days includes periods of rain, with temperatures generally ranging from the low single digits Celsius to the low double digits Celsius.


In [ ]:
response = ask("What's the weather in Mexico today?")


 ### Continue session: debug_session_id

User > What's the weather in Mexico today?
Rachet_assistant > The weather in Mexico today varies significantly by region.

In Mexico City, it is currently partly cloudy with a temperature of 57°F (14°C), feeling like 54°F (12°C). The wind is blowing from the North at 14 mph. The forecast for today in Mexico City suggests highs around 61°F (16°C) and lows around 45°F (7°C). There is a chance of rain showers in the evening. The air quality in Mexico City is currently rated as poor and unhealthy for sensitive groups.

In Cancun, the maximum daytime temperature is expected to be around 24°C (75°F).

For a broader overview, the country's high temperature is expected to be 88°F (31°C) in Hermosillo, and the low will be 45°F (7°C) in Ojinaga. The maximum wind speed across Mexico is predicted to be 22 mph in Querétaro.


In [ ]:
import mimetypes
mimetypes.add_type("text/javascript", ".js", True)

In [ ]:
!curl -sS "https://generativelanguage.googleapis.com/v1beta/models?key=$GOOGLE_API_KEY" \
 | grep -Eo '"name":\s*"models/[^"]+"' \
 | head -n 50

"name": "models/gemini-2.5-flash"
"name": "models/gemini-2.5-pro"
"name": "models/gemini-2.0-flash"
"name": "models/gemini-2.0-flash-001"
"name": "models/gemini-2.0-flash-exp-image-generation"
"name": "models/gemini-2.0-flash-lite-001"
"name": "models/gemini-2.0-flash-lite"
"name": "models/gemini-exp-1206"
"name": "models/gemini-2.5-flash-preview-tts"
"name": "models/gemini-2.5-pro-preview-tts"
"name": "models/gemma-3-1b-it"
"name": "models/gemma-3-4b-it"
"name": "models/gemma-3-12b-it"
"name": "models/gemma-3-27b-it"
"name": "models/gemma-3n-e4b-it"
"name": "models/gemma-3n-e2b-it"
"name": "models/gemini-flash-latest"
"name": "models/gemini-flash-lite-latest"
"name": "models/gemini-pro-latest"
"name": "models/gemini-2.5-flash-lite"
"name": "models/gemini-2.5-flash-image"
"name": "models/gemini-2.5-flash-preview-09-2025"
"name": "models/gemini-2.5-flash-lite-preview-09-2025"
"name": "models/gemini-3-pro-preview"
"name": "models/gemini-3-flash-preview"
"name": "models/gemini-3-pro-image

In [ ]:
!fuser -k 8000/tcp || true
!pkill -f "adk web" || true
!pkill -f "uvicorn" || true

^C
^C


In [ ]:
!mkdir -p /content/sample_agent/agents
%cd /content/sample_agent/agents
!pwd

/content/sample_agent/agents
/content/sample_agent/agents


In [ ]:
!rm -rf sample_agent
!adk create sample_agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY


Agent created in /content/sample_agent/agents/sample_agent:
- .env
- __init__.py
- agent.py



In [ ]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /usr/local/bin/cloudflared
!chmod +x /usr/local/bin/cloudflared
!nohup adk web --host 0.0.0.0 --port 8000 . > adk_web.log 2>&1 &
!cloudflared tunnel --url http://127.0.0.1:8000

2026-02-05T15:33:34Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2026-02-05T15:33:34Z INF Requesting new quick Tunnel on trycloudflare.com...
2026-02-05T15:33:38Z INF +--------------------------------------------------------------------------------------------+
2026-02-05T15:33:38Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2026-02-05T15:33:38Z INF |  https://assist-stay-fluid-amazing.trycloudflare.com  